https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence

In [186]:
%run pair_sequnce.ipynb

In [187]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import * 

from tensorflow.data import *

import numpy as np

In [188]:
user_number = 170
user_dim = 128

product_number = 52
product_dim = 64

class_dim = 2

In [193]:
def build_pair_model():

    user_input = layers.Input(shape = (user_dim,))
    x_1 = layers.Dense(64, activation='relu')(user_input)

    product_input = layers.Input(shape = (product_dim,))
    x_2 = layers.Dense(64, activation='relu')(product_input)

    x_3 = layers.Dot(axes=1)([x_1, x_2])
    x_3 = layers.Activation('sigmoid')(x_3)

    model = Model(
        inputs = [user_input, product_input],
         outputs = x_3,
        )

    model.compile(
        optimizer = 'adadelta',
        loss = 'binary_crossentropy',
        metrics = [keras.metrics.AUC()],
        )

    return model

In [189]:
user_feature = np.random.rand(
    user_number,
    user_dim
    )

product_feature = np.random.rand(
    product_number,
    product_dim
    )

user_product_pair_idx = np.array([
    [u_id, p_id]
    for p_id in range(product_number)
    for u_id in range(user_number)
    ])

user_product_pair_label = np.random.randint(
    class_dim,
    size = (len(user_product_pair), )
    )
#user_product_label = to_categorical(user_product_label)

In [190]:
x_user = user_feature[user_product_pair_idx[:,0]]
x_product = product_feature[user_product_pair_idx[:,1]]
x = [x_user, x_product]

y = user_product_pair_label #np.expand_dims(user_product_label, axis = 1)

In [191]:
y.shape

(8840,)

In [192]:
x_product.shape

(8840, 64)

In [194]:
model1 = build_pair_model()

model1.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 dense_46 (Dense)               (None, 64)           8256        ['input_1[0][0]']                
                                                                                                  
 dense_47 (Dense)               (None, 64)           4160        ['input_2[0][0]']                
                                                                                           

In [195]:
model1.fit(x, y)

277/277 [==============================] - 1s 3ms/step - loss: 1.8064 - auc_7: 0.5011


In [196]:
score = model1.predict(x)

277/277 [==============================] - 0s 848us/step


In [197]:
score.shape

(8840, 1)

https://www.tutorialsteacher.com/python/python-generator#:~:text=Python%20provides%20a%20generator%20to,rather%20than%20a%20return%20statement.

https://www.tensorflow.org/api_docs/python/tf/keras/utils/Sequence

In [199]:
class pair_sequnce(
    Sequence,
    ):
    
    def __init__(
        self, 
        batch_size,
        batch_number,
        x_user, 
        x_product,
        user_product_pair_idx,
        user_product_pair_label,        
        training = True,
        ):
        
        self.batch_size = batch_size
        self.batch_number = batch_number
        
        self.x_user = x_user
        self.x_product = x_product
        self.user_product_pair_idx = user_product_pair_idx
        self.user_product_pair_label = user_product_pair_label
        
        self.training = training

    def __len__(self):
        
        return int(self.batch_number)

    def __getitem__(self, batch_idx):
        
        start_idx = batch_idx*self.batch_size
        end_idx = start_idx+self.batch_size
        
        sample_idxs = range(start_idx, end_idx, 1)
        
        print(f' {batch_idx}-th batch: {start_idx} , {end_idx}')
        
        user_idx = self.user_product_pair_idx[sample_idxs,0]
        product_idx = self.user_product_pair_idx[sample_idxs,1]
              
        batch_x = [
            self.x_user[user_idx],
            self.x_product[product_idx],            
            ]
        
        batch_y = user_product_pair_label[sample_idxs]
        
        return batch_x, batch_y
    
    def on_epoch_end(
        self,
        ):
        #print('one epoch completed')
        pass

In [200]:
8840/100

88.4

In [201]:
sequence = pair_sequnce(
    batch_size = 100,
    batch_number = 88,
    x_user = x_user, 
    x_product = x_product,
    user_product_pair_idx = user_product_pair_idx,
    user_product_pair_label = user_product_pair_label,  
    )

In [202]:
sequence[12][0]

 12-th batch: 1200 , 1300


[array([[0.27853502, 0.19140026, 0.48099466, ..., 0.53051353, 0.95167895,
         0.32237259],
        [0.24698036, 0.39669892, 0.58446892, ..., 0.43695622, 0.46385134,
         0.29179008],
        [0.61652213, 0.75197021, 0.18031171, ..., 0.20288201, 0.4061025 ,
         0.16086999],
        ...,
        [0.48708832, 0.07956152, 0.1420983 , ..., 0.67645185, 0.57357789,
         0.85472923],
        [0.69726274, 0.83830563, 0.83025708, ..., 0.7316143 , 0.40063499,
         0.26781842],
        [0.64890317, 0.7735929 , 0.27064333, ..., 0.69443575, 0.6028189 ,
         0.65353549]]),
 array([[0.15271963, 0.13607665, 0.60005171, ..., 0.49334316, 0.45801788,
         0.50483285],
        [0.15271963, 0.13607665, 0.60005171, ..., 0.49334316, 0.45801788,
         0.50483285],
        [0.15271963, 0.13607665, 0.60005171, ..., 0.49334316, 0.45801788,
         0.50483285],
        ...,
        [0.15271963, 0.13607665, 0.60005171, ..., 0.49334316, 0.45801788,
         0.50483285],
        [0.1

In [203]:
sequence[12][1]

 12-th batch: 1200 , 1300


array([1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1])

In [204]:
model = build_pair_model()

model.fit(
    sequence, 
    epochs=10,
    batch_size=64,
    shuffle=True,
)

 0-th batch: 0 , 100
Epoch 1/10
 62-th batch: 6200 , 6300
 46-th batch: 4600 , 4700
 1/88 [..............................] - ETA: 27s - loss: 3.4282 - auc_8: 0.3943 28-th batch: 2800 , 2900
 31-th batch: 3100 , 3200
 71-th batch: 7100 , 7200
 22-th batch: 2200 , 2300
 0-th batch: 0 , 100
 78-th batch: 7800 , 7900
 50-th batch: 5000 , 5100
 60-th batch: 6000 , 6100
 57-th batch: 5700 , 5800
 43-th batch: 4300 , 4400
 52-th batch: 5200 , 5300
 25-th batch: 2500 , 2600
 77-th batch: 7700 , 7800
 76-th batch: 7600 , 7700
 70-th batch: 7000 , 7100
16/88 [====>.........................] - ETA: 0s - loss: 3.1747 - auc_8: 0.4919  79-th batch: 7900 , 8000
 29-th batch: 2900 , 3000
 13-th batch: 1300 , 1400
 68-th batch: 6800 , 6900
 16-th batch: 1600 , 1700
 30-th batch: 3000 , 3100
 11-th batch: 1100 , 1200
 47-th batch: 4700 , 4800
 33-th batch: 3300 , 3400
 39-th batch: 3900 , 4000
 51-th batch: 5100 , 5200
 49-th batch: 4900 , 5000
 6-th batch: 600 , 700
 19-th batch: 1900 , 2000
 21-th bat

82/88 [==========================>...] - ETA: 0s - loss: 2.8897 - auc_8: 0.5054 21-th batch: 2100 , 2200
 57-th batch: 5700 , 5800
 80-th batch: 8000 , 8100
 74-th batch: 7400 , 7500
 52-th batch: 5200 , 5300
88/88 [==============================] - 0s 3ms/step - loss: 2.9015 - auc_8: 0.5052
Epoch 4/10
 77-th batch: 7700 , 7800
 61-th batch: 6100 , 6200
 1/88 [..............................] - ETA: 0s - loss: 3.0189 - auc_8: 0.5143 59-th batch: 5900 , 6000
 11-th batch: 1100 , 1200
 21-th batch: 2100 , 2200
 72-th batch: 7200 , 7300
 46-th batch: 4600 , 4700
 50-th batch: 5000 , 5100
 80-th batch: 8000 , 8100
 70-th batch: 7000 , 7100
 3-th batch: 300 , 400
 26-th batch: 2600 , 2700
 78-th batch: 7800 , 7900
 32-th batch: 3200 , 3300
 7-th batch: 700 , 800
 60-th batch: 6000 , 6100
 75-th batch: 7500 , 7600
 56-th batch: 5600 , 5700
 31-th batch: 3100 , 3200
 63-th batch: 6300 , 6400
 85-th batch: 8500 , 8600
 54-th batch: 5400 , 5500
21/88 [======>.......................] - ETA: 0s - 

81/88 [==========================>...] - ETA: 0s - loss: 2.6550 - auc_8: 0.5053 40-th batch: 4000 , 4100
 23-th batch: 2300 , 2400
 58-th batch: 5800 , 5900
 81-th batch: 8100 , 8200
 45-th batch: 4500 , 4600
 24-th batch: 2400 , 2500
88/88 [==============================] - 0s 3ms/step - loss: 2.6538 - auc_8: 0.5053
Epoch 7/10
 65-th batch: 6500 , 6600
 28-th batch: 2800 , 2900
 1/88 [..............................] - ETA: 1s - loss: 2.0516 - auc_8: 0.4365 63-th batch: 6300 , 6400
 27-th batch: 2700 , 2800
 3-th batch: 300 , 400
 76-th batch: 7600 , 7700
 83-th batch: 8300 , 8400
 84-th batch: 8400 , 8500
 70-th batch: 7000 , 7100
 81-th batch: 8100 , 8200
 4-th batch: 400 , 500
 25-th batch: 2500 , 2600
 35-th batch: 3500 , 3600
 56-th batch: 5600 , 5700
 33-th batch: 3300 , 3400
 37-th batch: 3700 , 3800
 22-th batch: 2200 , 2300
 74-th batch: 7400 , 7500
 68-th batch: 6800 , 6900
 32-th batch: 3200 , 3300
 59-th batch: 5900 , 6000
 53-th batch: 5300 , 5400
21/88 [======>...........

80/88 [==========================>...] - ETA: 0s - loss: 2.4077 - auc_8: 0.5002 61-th batch: 6100 , 6200
 15-th batch: 1500 , 1600
 50-th batch: 5000 , 5100
 48-th batch: 4800 , 4900
 70-th batch: 7000 , 7100
 18-th batch: 1800 , 1900
 69-th batch: 6900 , 7000
88/88 [==============================] - 0s 3ms/step - loss: 2.4001 - auc_8: 0.5007
Epoch 10/10
 69-th batch: 6900 , 7000
 73-th batch: 7300 , 7400
 1/88 [..............................] - ETA: 1s - loss: 2.2766 - auc_8: 0.3981 8-th batch: 800 , 900
 72-th batch: 7200 , 7300
 59-th batch: 5900 , 6000
 1-th batch: 100 , 200
 22-th batch: 2200 , 2300
 45-th batch: 4500 , 4600
 21-th batch: 2100 , 2200
 32-th batch: 3200 , 3300
 33-th batch: 3300 , 3400
 43-th batch: 4300 , 4400
 49-th batch: 4900 , 5000
 65-th batch: 6500 , 6600
 74-th batch: 7400 , 7500
 36-th batch: 3600 , 3700
 13-th batch: 1300 , 1400
 56-th batch: 5600 , 5700
 38-th batch: 3800 , 3900
 4-th batch: 400 , 500
 58-th batch: 5800 , 5900
20/88 [=====>..............

In [180]:
score = model.evaluate(sequence)

 0-th batch: 0 , 100
 0-th batch: 0 , 100
 1-th batch: 100 , 200
 1/88 [..............................] - ETA: 2s - loss: 0.6956 - auc_6: 0.5086 2-th batch: 200 , 300
 3-th batch: 300 , 400
 4-th batch: 400 , 500
 5-th batch: 500 , 600
 6-th batch: 600 , 700
 7-th batch: 700 , 800
 8-th batch: 800 , 900
 9-th batch: 900 , 1000
 10-th batch: 1000 , 1100
 11-th batch: 1100 , 1200
 12-th batch: 1200 , 1300
 13-th batch: 1300 , 1400
 14-th batch: 1400 , 1500
 15-th batch: 1500 , 1600
 16-th batch: 1600 , 1700
 17-th batch: 1700 , 1800
 18-th batch: 1800 , 1900
 19-th batch: 1900 , 2000
 20-th batch: 2000 , 2100
20/88 [=====>........................] - ETA: 0s - loss: 0.6930 - auc_6: 0.5220 21-th batch: 2100 , 2200
 22-th batch: 2200 , 2300
 23-th batch: 2300 , 2400
 24-th batch: 2400 , 2500
 25-th batch: 2500 , 2600
 26-th batch: 2600 , 2700
 27-th batch: 2700 , 2800
 28-th batch: 2800 , 2900
 29-th batch: 2900 , 3000
 30-th batch: 3000 , 3100
 31-th batch: 3100 , 3200
 32-th batch: 3200 ,

In [181]:
score = model.predict(sequence)

 0-th batch: 0 , 100
 0-th batch: 0 , 100
 1-th batch: 100 , 200
 1/88 [..............................] - ETA: 2s 2-th batch: 200 , 300
 3-th batch: 300 , 400
 4-th batch: 400 , 500
 5-th batch: 500 , 600
 6-th batch: 600 , 700
 7-th batch: 700 , 800
 8-th batch: 800 , 900
 9-th batch: 900 , 1000
 10-th batch: 1000 , 1100
 11-th batch: 1100 , 1200
 12-th batch: 1200 , 1300
 13-th batch: 1300 , 1400
 14-th batch: 1400 , 1500
 15-th batch: 1500 , 1600
 16-th batch: 1600 , 1700
 17-th batch: 1700 , 1800
 18-th batch: 1800 , 1900
 19-th batch: 1900 , 2000
 20-th batch: 2000 , 2100
 21-th batch: 2100 , 2200
 22-th batch: 2200 , 2300
 23-th batch: 2300 , 2400
 24-th batch: 2400 , 2500
 25-th batch: 2500 , 2600
 26-th batch: 2600 , 2700
 27-th batch: 2700 , 2800
 28-th batch: 2800 , 2900
 29-th batch: 2900 , 3000
 30-th batch: 3000 , 3100
 31-th batch: 3100 , 3200
 32-th batch: 3200 , 3300
 33-th batch: 3300 , 3400
 34-th batch: 3400 , 3500
 35-th batch: 3500 , 3600
 36-th batch: 3600 , 3700


In [205]:
score.shape

(8840, 1)

In [213]:
score

array([[0.9842418],
       [0.9949882],
       [0.9918297],
       ...,
       [0.9306685],
       [0.9349595],
       [0.9064328]], dtype=float32)

In [214]:
user_product_pair_idx.shape

(8840, 2)

# end